# Табличные данные и взаимодействие с Excel

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Табличные данные и взаимодействие с Excel"
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.org/github/pybokeh/jupyter_notebooks/blob/main/xlwings/Excel_Formatting.ipynb
* https://xlsxwriter.readthedocs.io/
* https://xlsxwriter.readthedocs.io/working_with_conditional_formats.html
* https://xlsxwriter.readthedocs.io/example_conditional_format.html
* https://ru.excelfunctions.eu/
* https://docs.python.org/3/library/csv.html
* https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot
* https://analysistabs.com/excel-vba/colorindex/

## Задачи для совместного разбора

In [2]:
pip install xlsxwriter

     -------------------------------------- 150.0/150.0 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import xlsxwriter
import xlwings as xw
import numpy as np
import pandas as pd
import csv

1. Создать файл `demo.xlsx` при помощи пакета `xlswriter` на основе данных из файла `costs.csv`.

2. На листе "Рецептура" файла `demo.xlsx` рассчитать себестоимость всех видов продукции. Результаты расчетов сохранить в отдельном столбце.

3. Выполнить 2 с помощью "протягиваемых" формул.

4. Добавить общий заголовок и доработать форматирование новых столбцов

## Лабораторная работа 4

1. При помощи пакета `xlsxwriter` создайте рабочую книгу под названием `recipes_model.xlsx`. 
Добавьте лист с названием "Модель". В первой строчке листа разместите следующие названия колонок: `"Сущность", "Атрибут", "Тип данных", "Обязательно к заполнению", "Описание сущности", "Описание атрибута", "Ключ", "Таблица для ссылки", "Поле для ссылки", "SQL"`.

* Не закрывайте рабочую книгу, она потребуется для выполнения следующего задания.

In [12]:
workbook = xlsxwriter.Workbook('recipes_model.xlsx')
worksheet = workbook.add_worksheet(name="Модель")
  
content = ["Сущность", "Атрибут", "Тип данных", "Обязательно к заполнению", "Описание сущности", "Описание атрибута", "Ключ", "Таблица для ссылки", "Поле для ссылки", "SQL"]

worksheet.write_row(0, 0, content)

0

2. При помощи пакета `xlsxwriter` добавьте условное форматирование на лист "Модель":
* если в ячейке столбца `G` указано значение "PK", то сделать заливку ячейки цвета  `#006600`, а цвет шрифта - `#fffff`.
* если в ячейке столбца `G` указано значение "FK", то сделать заливку ячейки цвета  `#cccc00`, а цвет шрифта - `#000000`.

Примените данные правила к диапазону `G2:G50`. Закройте рабочую книгу.

In [13]:
with open('recipes_model.csv', 'r', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    data = list(reader)
    data[0][6]="Ключ"

In [15]:
format1 = workbook.add_format({"bg_color": "#006600", "font_color": "#fffff"})
format2 = workbook.add_format({"bg_color": "#cccc00", "font_color": "#000000"})

worksheet.conditional_format('G2:G50', {'type':     'cell',
                                        'criteria': '=',
                                        'value':    '"PK"',
                                        'format':   format1})
worksheet.conditional_format('G2:G50', {'type':     'cell',
                                        'criteria': '=',
                                        'value':    '"FK"',
                                        'format':   format2})

workbook.close()

3. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [16]:
def readBookAndSheets(Book: str, Sheets: str) -> xw.Sheet:
    global wb, recipies
    wb = xw.Book(Book)
    recipies = wb.sheets[Sheets]
    return recipies
readBookAndSheets("recipes_model.xlsx", "Модель")

<Sheet [recipes_model.xlsx]Модель>

In [18]:
try:
    model = wb.sheets.add(name="Модель")
except ValueError:
    model = wb.sheets["Модель"]

In [40]:
model.range("A2").value = data[1:]


In [ ]:
# wb.close()

4. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
# =ОБЪЕДИНИТЬ(" ";ИСТИНА;B2;ПРОПИСН(C2);ЕСЛИМН(G2="PK";ЕСЛИ(D2="Y";"PRIMARY KEY NOT NULL";"PRIMARY KEY");G2="FK";"REFERENCES "&ОБЪЕДИНИТЬ("";ИСТИНА;H2;"(";I2;")");G2="";""))
# =TEXTJOIN(" ";TRUE;B2;UPPER(C2);IFS(G2="PK";IF(D2="Y";"PRIMARY KEY NOT NULL";"PRIMARY KEY");G2="FK";"REFERENCES "&TEXTJOIN("";TRUE;H2;"(";I2;")");G2="";""))

In [41]:
from xlwings.constants import AutoFillType
SQL_column_start = model.range("J2")
SQL_column = model.range(f"J2:j{len(data)+1}")
SQL_column_start.formula = '=TEXTJOIN(" ",TRUE,B2,UPPER(C2),IFS(G2="PK",IF(D2="Y","PRIMARY KEY NOT NULL","PRIMARY KEY"),G2="FK","REFERENCES "&TEXTJOIN("",TRUE,H2,"(",I2,")"),G2="",""))'
SQL_column_start.api.AutoFill(SQL_column.api, AutoFillType.xlFillDefault)

True

5. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [43]:
Titles = model.range("A1:J1")
Titles.color = ("#00ccff")


6. В данных, полученных из файла `recipes_model.csv`, имеются ошибки двух видов:

    1\. Для некоторых полей, указанных как первичный ключ, в столбце "Обязательно к заполнению" стоит значение "N";

    2\. Для некоторых полей, указанных как внешний ключ, не указана таблица для ссылки и поле для ссылки.

Получите данные из листа в виде `pd.DataFrame`. Найдите строки на листе, которые содержат ошибки первого или второго рода и сделайте заливку эти строк цвета `(255, 22, 22)`. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

7. При помощи пакета `xlwings` создайте лист с названием `Ошибки заполнения`. 

Поместите в первой строчке этого листа заголовки "№ строки" и "Ошибка". Заполните лист на основе решения задачи 6. В столбце "№ строки" укажите номер строки с листа `Модель`, содержащую ошибку, а в столбце "Ошибка" для ошибок первого рода напишите сообщение "Первичный ключ не может быть NULL", для ошибок второго рода - "Не указано поле для связи по внешнему ключу". Сделайте скриншот состояния листа `Ошибки заполнения` и прикрепите в ячейку ноутбука. 

8. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 